In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# pandas version
print('pandas',pd.__version__)

In [ ]:
# define train set
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('./drive/My Drive/DACON/data/train.csv')

In [ ]:
train_img = train.copy()

# reshape and normalization
x_28 = np.divide(train_img.iloc[:,3:].values.copy(),255).reshape(-1,28,28,1)
x_train = np.zeros((x_28.shape[0],112,112,1))

plt.figure(figsize = (10,10))
plt.subplot(1,2,1)
plt.imshow(x_28[0].reshape(28,28),cmap='gray')

npad = ((42,42),(42,42))
for i in range(x_28.shape[0]):
    x_train[i] = np.pad(x_28[i].reshape(28,28),npad,'constant',constant_values=(0)).reshape(112,112,1)

plt.subplot(1,2,2)
plt.imshow(x_train[0].reshape(112,112),cmap='gray')

# label encoding
y_28 = train_img.iloc[:,3:].copy()

train_copy = train.copy()
train_img_avg = np.zeros((10,26,784))

for i in range(10):
    for j in range(26):
        a = train_copy[(train_copy['digit']==i) & (train_copy['letter']==chr(j+65))]
        b = a.iloc[0,3:]
        for k in range(1,len(a)-1):
            b += a.iloc[k,3:] 
        b = np.divide(b,len(a))
        train_img_avg[i][j] = b

for i in range(train_img.shape[0]):
    y_28.iloc[i,:] = train_img_avg[train_img.iloc[i,1]][ord(train_img.iloc[i,2])-65] 
y_28 = np.divide(y_28.values,255).reshape(-1,28,28,1)
y_train = np.zeros((y_28.shape[0],112,112,1))

plt.figure(figsize = (10,10))
plt.subplot(1,2,1)
plt.imshow(x_28[0].reshape(28,28),cmap='gray')

npad = ((42,42),(42,42))
for i in range(y_28.shape[0]):
    y_train[i] = np.pad(y_28[i].reshape(28,28),npad,'constant',constant_values=(0)).reshape(112,112,1)

plt.subplot(1,2,2)
plt.imshow(y_train[0].reshape(112,112),cmap='gray')

# split training and validation set
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=15)

In [ ]:
# set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

def unet(input_size = (112,112,1)):
    inputs = Input(input_size)
    N = 16
    conv1 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(16*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(16*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(8*N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(4*N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(2*N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = "binary_crossentropy", metrics = ['accuracy'])
    
    # model.summary()

    # if(pretrained_weights): model.load_weights(pretrained_weights)

    return model

In [ ]:
# with data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

# set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

# fit the model

epochs = 5
batch_size = 50

model_u = unet()
history = model_u.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, 
                              validation_data = (x_val,y_val),
                              steps_per_epoch=x_train.shape[0]// batch_size
                              ,callbacks=[learning_rate_reduction]
                             )

# plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
z = np.divide(train.copy().iloc[:,3:].values,255).reshape(-1,28,28,1)
w = np.zeros((z.shape[0],112,112,1))

npad = ((42,42),(42,42))
for i in range(z.shape[0]):
    w[i] = np.pad(z[i].reshape(28,28),npad,'constant',constant_values=(0)).reshape(112,112,1)
ip =  np.divide(w[1],255).reshape(1,112,112,1)
print(ip.shape)
plt.imshow(ip.reshape(112,112),cmap='gray')

In [ ]:
op = model_u.predict(ip)
plt.imshow(op.reshape(112,112),cmap='gray')